## **Setup**

### Conda-Forge

- **Installazione**

https://conda-forge.org/

In caso non si volesse avere l'enviroment già attivato nel terminale:
```bash
conda config --set auto_activate_base false
```

- **Creazione nuovo enviroment**

    ```bash
    conda create --name llm_env python=3.10
    ```

- **Attivazione enviroment**

    ```bash
    conda activate llama_env
    ```

- **Disattivazione enviroment**

    ```bash
    conda deactivate llama_env
    ```

### Installazione dipendenze progetto

- **pip**

    ```bash
    conda install pip
    ```


- **[Llama Index](https://www.llamaindex.ai/)**

    ```bash
    pip install llama-index
    ```

    ```bash
    pip install llama-index-llms-ollama

    ```


- **[Gradio](https://www.gradio.app/)**
    ```bash
    pip install gradio
    ```

## [Ollama](https://ollama.com/)

### Install models

```bash
ollama pull <model:version>
```

**Models**

```bash
ollama pull llama3.1:8b
```

```bash
# versione da usare per provare gli agenti e i tool
ollama pull llama3.2:3b
```

```bash
# versione da usare per le lezioni
ollama pull llama3.2:1b
```

```bash
ollama pull llama3.1:8b-instruct-q4_0
```

### Run models

```bash
ollama run <model:version>
```


```bash
ollama serve <model:version>
```

### Remove models

```bash
ollama rm <model:version>
```

## **Simple Chat**

https://docs.llamaindex.ai/en/stable/api_reference/llms/ollama/

In [2]:
from llama_index.llms.ollama import Ollama

In [3]:

def simple_chat(question: str = "", temperature: float = 0., top_p: float = 0.):
    """
    Args:

        @ question (string): passa il testo immesso da un utente al modell.

        @ temperature (float): incide sulla capacità di un llm di scegliere l
                               la successione di parole.

    Return:
       @ (string): risposta del modello linguistico.
    """



    model = Ollama(model="llama3.2:1b", 
                   temperature=temperature,
                   top_p=top_p)


    return model.complete(question)

# **Tecniche di Prompt Engineering**

Le tecniche di prompt engineering si concentrano su vari aspetti, come la formulazione chiara e concisa della domanda, l'inclusione di contesti specifici, la strutturazione del prompt in modo che il modello comprenda correttamente l'intento dell'utente, e l'uso di parametri come la temperatura o la lunghezza della risposta. Un buon prompt può migliorare significativamente la qualità delle risposte ottenute, mentre un prompt mal strutturato può portare a risultati imprecisi o incoerenti.

Le principali tecniche di prompt engineering includono:

- **Prompt chiari e diretti**: Evitare ambiguità e assicurarsi che la richiesta sia comprensibile.
- **Contesto e dettagli**: Fornire informazioni aggiuntive per indirizzare il modello verso risposte più pertinenti.
- **Format specifico**: Utilizzare istruzioni che richiedano una risposta in un formato definito, come liste o paragrafi.
- **Temperature e altre variabili**: Regolare parametri come la temperatura per influenzare la creatività o la precisione delle risposte.
- **Prompt iterativi**: Affinare il prompt attraverso tentativi successivi, per affinare la risposta.

## **L'uso della negazione**

Non sembra essere una buona pratica, quella di scrivere esplicitamente cosa ***NON deve fare un LLM***.

```
Don't say ... 
Don't do ...
```

Le espressioni legate alla negazione esplictia possono essere legate a contesti e finalità differenti. 
L'effetto finale, potrebbe quindi essere quello di determinare una serie di ambiguità nella definizione dei pattern all'interno di un llm, tali da aumentare la probabilità di risultati poco coerenti.


```
Don't use these words: ... 
```

```
You don't have this type of item at your disposal: ...
```



Usare in alternativa espressioni univoche,  accompagnate da una spiegazione.

```
Avoid saying these words: ...
```

La stessa forma, utilizzata per esprimere una quantità non avrebbe senso o per dirla come un llm:
la probabilità che sia presente nel dataset di addestramento, è molto poco probabile.

```
Avoid having these items available: ...
```


N.B | Quanto detto vale non solo per il prompt engineering, ma anche nell'uso quotidiano di un llm.

## **In-Context Learning (ICL)** 



Si tratta di un concetto centrale nei modelli linguistici avanzati, come GPT,  si riferisce alla capacità del modello di apprendere e adattarsi a un compito specifico fornendo esempi o istruzioni direttamente nel prompt, senza dover essere ulteriormente addestrato o fine-tuned.


- **Uso del contesto**

Il modello utilizza il testo fornito nel prompt come guida per generare risposte o completare un compito. Gli esempi servono come "istruzioni implicite" per definire il comportamento atteso.

- **Formati di prompting**

    - **Zero-shot**: Nessun esempio, solo istruzioni.
    - **One-shot**: Un esempio per chiarire il formato o il compito.
    - **Few-shot**: Più esempi per fornire un contesto più completo.
    

- **Adattabilità senza riaddestramento**

A differenza dei tradizionali metodi di apprendimento, in cui il modello viene addestrato con nuovi dati, l'ICL sfrutta la conoscenza preesistente del modello, combinandola con le informazioni fornite nel prompt.

### **Zero-shot prompting**

Non si fornisce alcun esempio al modello. Gli si chiede semplicemente di rispondere a una domanda o completare un compito basandosi solo sulla comprensione implicita del linguaggio.


In [3]:
ZERO_SHOT = \
"""
<INSTRUCTION>
Classify the text into neutral, negative or positive. 

<CHAT>
User: I think the vacation is okay.
Assistant:
"""

print(simple_chat(ZERO_SHOT, 0, 0))

I would classify the text as neutral. The user's statement "I think the vacation is okay" does not express a strong emotion such as positivity or negativity. It simply states a factual opinion without any emotional tone.


### **One/Few-shot prompting**


Si forniscono alcuni esempi (1-5) per rafforzare la comprensione del modello rispetto alla natura del compito.


In [4]:


ONE_SHOT = \
"""
<INSTRUCTION>
Classify the text into neutral, negative or positive. 

<EXAMPLES>
User: This is awesome! 
Assistant: Positive 
User: This is bad! 
Assistant: Negative
User: Wow that movie was rad!
Assistatn: Positive


<CHAT>
User: What a horrible show! 
Answare: 
"""

print(simple_chat(ONE_SHOT, 0., 0.))

I can classify the text as follows:

- User: "What a horrible show!" - Neutral (the user's opinion is subjective and not necessarily positive or negative)
- Assistant: "This is bad!" - Negative (the assistant's response is direct and expresses disapproval)


## **Chain-of-Thought (CoT) Prompt**

[Kojima et al. (2022)](https://arxiv.org/abs/2205.11916)

La tecnica di ***Chain-of-Thought (CoT)**** è un metodo di prompting progettato per migliorare le capacità di ragionamento dei modelli linguistici. Consiste nel guidare il modello attraverso una sequenza esplicita di passaggi intermedi per risolvere un problema complesso, anziché richiedere direttamente la risposta finale. Questo approccio incoraggia il modello a "pensare passo dopo passo" e a simulare un processo di ragionamento simile a quello umano.

La maggior parte dei modelli attuali è stato già addestrato per usare questa tecnica in autonomia. 
Se l'uso di questo approccio non è contenuto in modo esplicito dal contesto, può essere "attivato" 
da espressioni come:

```
proceed step by step
```

Questa espressione, utilizzata direttamente, permette anche di comprendere in che modo costruire un "ragionamento" esplicito, a partire dal tipo di output prodotto.

In [5]:
BASE_PROMPT = \
"""
Tom used a pice of wire 10cm long to support tomato plants in the garden.
The wire was cut into 6 pices. 
How long is each individual piece of wire?
"""

print(simple_chat(BASE_PROMPT, 0., 0.))

To find out how long each individual piece of wire is, we need to divide the total length of the wire (10 cm) by the number of pieces it was cut into (6).

10 cm ÷ 6 = 1.67 cm per piece

So each individual piece of wire is approximately 1.67 cm long.


In [6]:
SBS_PROMPT = \
BASE_PROMPT + "Proceed step-by-step"

print(simple_chat(SBS_PROMPT, 0., 0.))

To find out how long each individual piece of wire is, we need to divide the total length of the wire by the number of pieces it was cut into.

Step 1: Find the total length of the wire
The total length of the wire is given as 10cm.

Step 2: Divide the total length by the number of pieces
To find out how long each individual piece is, we divide the total length (10cm) by the number of pieces it was cut into (6).

10 cm ÷ 6 = 1.67 cm

So each individual piece of wire is approximately 1.67cm long.


## **ReAct** (Reason + Act).

[Yao et al.2022](https://arxiv.org/pdf/2210.03629.pdf)

La tecnica di ReAct (Reasoning + Acting) combina ragionamento logico e azioni interattive nei prompt, per permettere ai modelli linguistici di risolvere problemi complessi che richiedono sia pensiero deduttivo che interazioni con l'ambiente (ad esempio, consultare un database o chiamare una funzione in un determinato linguaggio di programmazione).

In [4]:

from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent


In [ ]:

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b



def test_agent():


    llm = Ollama(model="llama3.2:1b")
    #llm = Ollama(model="llama3.2:3b")
    #llm = Ollama(model="llama3.1:8b")
    

    multiply_tool = FunctionTool.from_defaults(fn=multiply)
    add_tool = FunctionTool.from_defaults(fn=add)
    subtract_tool = FunctionTool.from_defaults(fn=subtract)
    divide_tool = FunctionTool.from_defaults(fn=divide)


    agent = ReActAgent.from_tools(
        [multiply_tool, add_tool, subtract_tool, divide_tool],
        llm=llm,
        verbose=True,
    )


    response = agent.chat("What is (121 + 2) * 5?")
    print(str(response))

test_agent()